# Data Quality Analysis as a Precursor to Machine Learning Activities

A student-information-system (SIS) intends to include additional ML-based insights into its management reporting suite. One of the csv data files used to prime the SIS Machine Learning application has been found to have significant data quality issues.

You have been asked to provide advice and guidance on the steps that need to be taken to clean the data file as part of quality assurance for the ML system.

**To complete this assignment:**

Carry out the following tasks

1. Perform detailed analysis of data quality for the dataset provided, making use of some of the criteria discussed in the week’s lectures.
2. Map out, in outline form, what data cleansing measures would need to be put in place to prepare the dataset for ML work. Indicate the types of Pandas-Scikit-Learn python commands that could be used whilst carrying out the measures you suggested in task 2.
3. Attempt a clean-up of the dataset using the procedures set out in (2) and (3) and report on the degree of success with carrying this out.
4. Submit the final document (in MS-Word or pdf format) containing your responses for sub-tasks 1, 2, 3 and 4 in form of a brief report (500 words). The Python code must be included in your submission.

## Importing and Data Preparation

### Importing

This project uses `pandas` and `numpy` in order to cleanse the provided dataset. It also utilises the `re` package and `Counter` from the `collections` package. Finally, the `sklearn` package allows us to preprocess the cleaned data for a hypothetical ski-kit model. This project starts by importing these packages and importing the raw dataset into a pandas DataFrame.

In [1]:
# Import Packages
import numpy as np
import pandas as pd
import re
from collections import Counter
from sklearn import preprocessing

# Import Data
data = pd.read_csv('source_data\SIS_Faculty-List.csv', encoding='utf-8')

### Initial Data Preparation
Some of the column names are long or contain new lines, so these are removed.

A common data quality issue can be leading or trailing spacaes, so as a precaution for the columns made of strings, the columns are trimmed to remove these.

In [2]:
# Data Preparation
# Remove new lines from column names
data.columns = data.columns.str.replace('\n', ' ')
# Update Long Column Name
data.columns = data.columns.str.replace('DOCUMENT OTHER PROFESSIONAL '
                                        'CERTIFICATION CRITIERA '
                                        'Five Years Work '
                                        'Experience Teaching '
                                        'Excellence Professional '
                                        'Certifications',
                                        'Other Experience')
data.columns = data.columns.str.replace('Courses Taught- '
                                        'Term 201510',
                                        'Courses Taught')
data.columns = data.columns.str.replace('All Qualifications '
                                        'from Profile',
                                        'All Qualifications')


# Trim all remaining columns that are strings
data['Name'] = data['Name'].str.strip()
data['Location'] = data['Location'].str.strip()
data['Title'] = data['Title'].str.strip()
data['Reports To'] = data['Reports To'].str.strip()
data['Highest Qualification'] = data['Highest Qualification'].str.strip()
data['Major'] = data['Major'].str.strip()
data['University'] = data['University'].str.strip()
data['All Qualifications'] = data['All Qualifications'].str.strip()
data['Courses Taught'] = data['Courses Taught'].str.strip()
data['MAJOR TEACHING FIELD'] = data['MAJOR TEACHING FIELD'].str.strip()
data['Other Experience'] = data['Other Experience'].str.strip()
data['Criteria'] = data['Criteria'].str.strip()

### Examining the Data

In order to determine the steps to be taken with the dataset, the data first needs to be examined.

In [3]:
data.head()

,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Divison,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria
0,LT9210743,Jim Hunter,London,FA,Faculty - Business,29-Aug-10,NaN,Full Time,Business,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,LT9001784,Jose Nadal,London,FA,Faculty - Business,19-Aug-99,NaN,Full Time,Business,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,LT9062955,Pierre Cornet,London,FA,Faculty - Business,08-Apr-07,NaN,Full Time,Business,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,0,Sally Smith,London,FA,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,0,Ned Shearer,London,Chair,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


Initial conclusions from viewing the dataset
- There are likely vital missing values and potential duplicates
- There are quite a few columns with multiple pieces of information
- There are multiple columns that contain similar data for qualifications

#### Unique Identifier

Firstly the `['ID']` column is checked for unique identification.

In [4]:
# Check for duplicates
print(data['ID'].value_counts())

# Check for nulls
id_missing = data['ID'].isnull().sum()
print("\nNumber of missing values:", id_missing)

0            40
LT9210743     1
LT9017549     1
LT9001148     1
LT9305052     1
             ..
LT9308826     1
LT9210710     1
LT9303893     1
LT9328836     1
LT9307987     1
Name: ID, Length: 243, dtype: int64

Number of missing values: 2


Taking a first look at the columns shows the `['ID']` column has missing values and duplicate values where it is set to 0. There are no duplicate ids besides the ones that are missing / set to 0, and there are no duplicate names of these ids so it is safe to assume each row is unique.

A new unique id column is added to the dataset for the reference of this project. It should be recommended that the original ids are recovered in order to link back to the original dataset.

In [5]:
# Add a unique id column
u_id = data.index
data.insert(0, 'u_id', u_id)

data.head()

,u_id,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Divison,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria
0,0,LT9210743,Jim Hunter,London,FA,Faculty - Business,29-Aug-10,NaN,Full Time,Business,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,1,LT9001784,Jose Nadal,London,FA,Faculty - Business,19-Aug-99,NaN,Full Time,Business,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,2,LT9062955,Pierre Cornet,London,FA,Faculty - Business,08-Apr-07,NaN,Full Time,Business,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,3,0,Sally Smith,London,FA,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,4,0,Ned Shearer,London,Chair,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


## Data Analysis

### Values in the Columns
The contents of the columns are examinded in order to identify if any columns are initially redudant. This is done by counting the number of unique values in each column and eliminating those where most of the column is the same information.

In [6]:
# Get total number of rows
total = len(data)
print("Total Number of Rows", total)

# Summarise the data
data.describe(include='O')

Total Number of Rows 284


,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Divison,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria
count,282,284,270,284,284,283,11,284,284,280,278,278,262,278,274,273,225,222,284
unique,243,282,15,2,11,143,2,1,1,15,18,59,112,214,262,269,129,207,8
top,0,Zikida Koudou,Bristol,FA,Faculty - Business,14-Aug-16,17-Dec-15,Full Time,Business,Ian Varley,Masters,Master of Business Administrat,Business Administration,University of Wollongong,Master of Business Administrat (Business Admin...,Faculty Trainee/MBA in Progress,General Business,More than 5 years work experience,Industry & Teaching experience
freq,40,2,49,282,253,41,6,284,284,33,157,73,45,8,7,2,20,14,231


Columns with two or less values are examinded more closely.

In [7]:
# Print the counts for each of the values within the columns
print(data['Grade'].value_counts())
print(data['LWD'].value_counts())
print(data['Type'].value_counts())
print(data['Divison'].value_counts())

FA       282
Chair      2
Name: Grade, dtype: int64
17-Dec-15    6
22-Dec-15    5
Name: LWD, dtype: int64
Full Time    284
Name: Type, dtype: int64
Business    284
Name: Divison, dtype: int64


For `['Grade']`, `['Type']` and `['Divison']` the majority of the columns contain the same data, so are removed. `['LWD']` only have two dates and is mostly null otherwise, however it may be a column useful to filtering, such as if we only want data from active student, so this column will be kept for now.

In [8]:
# Drop irrelevant columns Grade, Type, Division
data = data.drop(columns=['Grade', 'Type', 'Divison'])

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


### Checking for missing values
The columns are then examinded for missing values. These results will be used throughout each individual examination of each column to address missing values during cleaning.

In [9]:
# Count Missing
count_missing = data.isnull().sum()
total = len(data)

percent_missing = (count_missing/total) * 100
print(percent_missing)

u_id                            0.000000
ID                              0.704225
Name                            0.000000
Location                        4.929577
Title                           0.000000
Join Date                       0.352113
LWD                            96.126761
Reports To                      1.408451
Highest Qualification Level     2.112676
Highest Qualification           2.112676
Major                           7.746479
University                      2.112676
All Qualifications              3.521127
Courses Taught                  3.873239
MAJOR TEACHING FIELD           20.774648
Other Experience               21.830986
Criteria                        0.000000
dtype: float64


## Extracting Information from Data

### Extracting useful information from Courses Taught
The `['Courses Taught']` column contains multiple values for courses. Having a column with multiple pieces of information inside of it is not very useful, but previous experience of specific subjects could be a useful machine learning feature. 

This column is transformed into a separate table which can be used alongside the main table in the future.

In [10]:
# Courses Table
# Split out the courses in new columns
courses_t = data['Courses Taught'].str.split(',|\n', expand=True)

# Ensure the unique id is added to the new dataframe
courses_t.insert(0, 'u_id', u_id)

# Transform the new dataframe so the columns become rows
courses_t = courses_t.melt(id_vars=['u_id'],
                           var_name='column',
                           value_name='Course')

# Remove empty rows
courses_t = courses_t[~courses_t['Course'].isnull()]

# Trim columns and remove redudnant column
courses_t['Course'] = courses_t['Course'].str.strip()
courses_t = courses_t.drop(columns=['column'])

# Export to a new file
courses_t.to_csv('cleaned_data\courses_table.csv')
courses_t.head()

,u_id,Course
0,0,BUS2403 - Innovation & Entrepreneurship
1,1,BUS4173 - Managerial Accounting II
2,2,BUS3903 - Contemporary Business
3,3,HRM
4,4,Micro and Macroeconomics


It is possible to extract information using the The `['Courses Taught']` column in order to indicate past teaching experience. This is done by finding the most common subject keywords and indicating if these keywords appear in this column for each student.

In [11]:
# Convert the column to a list of courses and split by new line and space
courses_list = data['Courses Taught'].to_list()
courses_list_keywords = []
for x in range(0, len(courses_list)-1):
    y = str(courses_list[x])
    courses_list_keywords.append(re.split(r' |\n', y))

# Count the occurance of each of the keywords
courses_list_keywords = [x for y in courses_list_keywords for x in y]
courses_list_keywords_count = Counter(courses_list_keywords)

# Get most common keywords
common = courses_list_keywords_count.most_common(20)
items = [x[0] for x in common]

for x in items:
    if x.isalpha():
        print(x)

Business
and
Management
Accounting
for
Work
Financial
Related
Learning
Finance
Marketing
Mgt
Managers
Strategic
Busi
Project
Corporate


Using this list, new columns are put together that could be useful for machine learning analysis. This would be dependant on the application for this dataset, and should only be considered an example.

The most popular keywords are chosen to create the columns `['t_business']`, `['t_management']`, `['t_account']`, `['t_finance']` and `['t_market']` for experience teaching Business, Management, Accounting, Finance and Marketing respectively.

In [12]:
# Clean Courses
# Adding a '1' for yes where the keyword exists in the courses taught column
data['t_business'] = np.where(data['Courses Taught'].str.contains('Business') |
                              data['Courses Taught'].str.contains('Busi '),
                              1, 0)

data['t_management'] = np.where(data['Courses Taught'].str.contains('Manag') |
                                data['Courses Taught'].str.contains('Mgmnt') |
                                data['Courses Taught'].str.contains('Mgt'),
                                1, 0)

data['t_account'] = np.where(data['Courses Taught'].str.contains('Accounting'),
                             1, 0)

data['t_finance'] = np.where(data['Courses Taught'].str.contains('Financ'),
                             1, 0)

data['t_market'] = np.where(data['Courses Taught'].str.contains('Marketing'),
                            1, 0)

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,All Qualifications,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria,t_business,t_management,t_account,t_finance,t_market
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification,0,1,1,0,0
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,"Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,"Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0


### Cleaning Title
Examining the contents of the `['Title']` column gives some inconsistencies in the formatting.

In [13]:
# Print the counts for each of the values within the columns
print(data['Title'].value_counts())

Faculty - Business                       253
Faculty - Trainee                         13
Adjunct Faculty - Business                 9
Faculty- Business                          2
Faculty - Business (Admin)                 1
Faculty-Business (HR & Admin.)             1
Faculty - Business & Comp Lit Faculty      1
Faculty-Business                           1
Business, Comp & Math Faculty              1
Graduate Trainee                           1
Faculty - Business & Comp Lit.             1
Name: Title, dtype: int64


These are corrected to give move consistent values. It is worth noting that this column is similarly not very diverse and may be considered redundant.

In [14]:
# Clean Title
data['Title_clean'] = np.where(data['Title'].str.contains('HR') &
                               data['Title'].str.contains('Business'),
                               'Business (HR & Admin)',
                      np.where(data['Title'].str.contains('Admin') &
                               data['Title'].str.contains('Business'),
                               'Business (Admin)',
                      np.where(data['Title'].str.contains('Comp') &
                               data['Title'].str.contains('Business') &
                               data['Title'].str.contains('Math'),
                               'Business, Comp & Math',
                      np.where(data['Title'].str.contains('Comp Lit') &
                               data['Title'].str.contains('Business'),
                               'Business & Comp Lit',
                      np.where(data['Title'].str.contains('Trainee'),
                               'Trainee',
                      np.where(data['Title'].str.contains('Business'),
                               'Business', 'Other'))))))

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,Courses Taught,MAJOR TEACHING FIELD,Other Experience,Criteria,t_business,t_management,t_account,t_finance,t_market,Title_clean
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0,Business
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification,0,1,1,0,0,Business
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1,Business
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,"HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0,Business
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,"Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0,Business


### Cleaning Highest Qualification
There are several columns that refer to the qualifications of the person. These are a bit inconsistent and could do with some tidying up.

The first is the `['Highest Qualification Level']` and `['Highest Qualification']`.

In [15]:
# Clean Highest Qualification
# Print the counts for each of the values within the columns
print(data['Highest Qualification Level'].value_counts())

# Print the counts for each of the values within the columns
print(data['Highest Qualification'].value_counts())

Masters                                                                     157
Doctorate                                                                    52
Ph.D                                                                         43
Bachelor                                                                     12
Masters (Business Administration), Master of Philosophy (Business)            1
 Doctorate of Business Administration                                         1
 Ph. D. (Business Administration)                                             1
Maters of Philosophy (Economics)                                              1
Master's Degree in Public Relations and Press Office Management               1
PhD Corporate Governance and Accounting                                       1
Master of Business Administration (MBA) and MA (Economics)                    1
Bachelor (Applied Science) in Business administration( Human Resources )      1
Ph.D in Organization and Management     

`['Highest Qualification Level']` shares values with the `['Highest Qualification']` column, as well as `['Highest Qualification']` containing lots of varied values for each individual subject, despite there being the column `['Major']` that should be storing this.

As shown previously, there are also some null values in the columns. We can take information from the `['All Qualifications from Profile']` column in order to attempt to backfill these null values.

Using the data in these columns they are cleaned up into more succinct categories.

In [16]:
# Clean Highest Qualification Level
data['HQL_clean'] = np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Ph.D'),
                             'Doctorate',
                    np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Master'),
                             'Masters',
                    np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Bachelor'),
                             'Bachelors',
                    np.where(data['Highest Qualification Level'].str.contains('Master'),
                    'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Ph.D') |
                             data['Highest Qualification Level'].str.contains('PhD'),
                             'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'),
                    'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor') &
                            (data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('philosophy')),
                             'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor'),
                    'Doctorate',
                    np.where(data['Highest Qualification'].str.contains('Doctor'),
                    'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('Bachelor'),
                    'Bachelors',
                             'Unknown'))))))))))

# Clean Highest Qualification
data['HQ_clean'] = np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Ph.D'), 'Ph.D',
                    np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Master'), 'Masters',
                    np.where(data['Highest Qualification Level'].isna() &
                             data['All Qualifications'].str.contains('Bachelor'), 'Bachelors',
                    np.where(data['Highest Qualification'].str.contains('Master of Arts'), 'Master of Arts',
                    np.where(data['Highest Qualification'].str.contains('Master of Science'), 'Master of Science',
                    np.where(data['Highest Qualification'].str.contains('Master of Business Administrat'), 'Master of Business Administration',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Applied Science'), 'Bachelor of Applied Science',
                    np.where(data['Highest Qualification'].str.contains('Master of Commerce'), 'Master of Commerce',
                    np.where(data['Highest Qualification'].str.contains('Master of Marketing'), 'Master of Marketing',
                    np.where(data['Highest Qualification'].str.contains('Master of Business'), 'Master of Business',
                    np.where(data['Highest Qualification'].str.contains('Master of Management'), 'Master of Management',
                    np.where(data['Highest Qualification'].str.contains('Master of Law'), 'Master of Law',
                    np.where(data['Highest Qualification'].str.contains('Master of Philosophy'), 'Master of Philosophy',
                    np.where(data['Highest Qualification'].str.contains('Master of Education'), 'Master of Education',
                    np.where(data['Highest Qualification'].str.contains('Master of International Business'), 'Master of International Business',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Science'), 'Bachelor of Science',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Business Admin'), 'Bachelor of Business Administration',
                    np.where(data['Highest Qualification'].str.contains('Master of Professional Studies'), 'Master of Professional Studies',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'), 'Master of Business Administration',
                    np.where(data['Highest Qualification Level'].str.contains('Master'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Ph.D') |
                             data['Highest Qualification Level'].str.contains('PhD') |
                             data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('PhD'), 'Ph.D',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor') &
                            (data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('philosophy')), 'Ph.D',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor'), 'Doctorate (Other)',
                    np.where(data['Highest Qualification'].str.contains('Doctor'), 'Doctorate (Other)',
                    np.where(data['Highest Qualification Level'].str.contains('Bachelor'), 'Bachelors',
                             'Unknown'))))))))))))))))))))))))))

# Get counts of values
print(data['HQL_clean'].value_counts())
print(data['HQ_clean'].value_counts())

Masters      168
Doctorate    102
Bachelors     13
Unknown        1
Name: HQL_clean, dtype: int64
Ph.D                                   93
Master of Business Administration      76
Masters                                40
Master of Science                      18
Doctorate (Other)                       9
Bachelor of Applied Science             9
Master of Commerce                      9
Master of Arts                          7
Master of Philosophy                    5
Master of Education                     3
Master of Management                    3
Master of Law                           2
Master of Business                      2
Bachelors                               2
Master of Marketing                     1
Master of International Business        1
Bachelor of Science                     1
Bachelor of Business Administration     1
Master of Professional Studies          1
Unknown                                 1
Name: HQ_clean, dtype: int64


Running the check again gives streamlined categories and only one remaining Unknown value. Since there is only one, this can be accounted for by filling it with the most common value, in this case `'Masters'`.

In [17]:
# Remove Unknown
data['HQL_clean'] = data['HQL_clean'].str.replace('Unknown', 'Masters', regex=False)
data['HQ_clean'] = data['HQ_clean'].str.replace('Unknown', 'Masters', regex=False)

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,Other Experience,Criteria,t_business,t_management,t_account,t_finance,t_market,Title_clean,HQL_clean,HQ_clean
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0,Business,Masters,Master of Arts
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,years of teaching experience: 17 ...,Certification,0,1,1,0,0,Business,Masters,Master of Science
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1,Business,Masters,Master of Business Administration
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0,Business,Doctorate,Ph.D
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0,Business,Doctorate,Ph.D


The `['Major']` column can then be used to store the subject. This could be cleaned and extracted from the various available columns and streamlined in a similar way above to indicate the staffs main subject expertise.

### All Qualifications from Profile

`['All Qualifications']` shows all qualifications and subjects in a relatively clean format but contains multiple pieces of information. It could be used to verify or populate the highest qualification available, although there are some discrepancies between the columns where some rows contain what is indicated as the highest and others do not.

However, `['All Qualifications']` can also be transformed similarly to the courses taught column in order to extract information that could be helpful in the future. This is demonstrated below.

In [18]:
# Qualifications Table
# Split out the courses in new columns
qualifs = data['All Qualifications'].str.split(',', expand=True)

# Ensure the unique id is added to the new dataframe
qualifs.insert(0, 'u_id', u_id)

# Transform the new dataframe so the columns become rows
qualifs_t = qualifs.melt(id_vars=['u_id'], var_name='column', value_name='qualif')

# Remove empty rows
qualifs_t = qualifs_t[~qualifs_t['qualif'].isnull()]

# Split out qualification from subject
qualifs_t[['Qualification', 'Major']] = qualifs_t['qualif'].str.split('(', 1, expand=True)

# Clean up
qualifs_t['Major'] = qualifs_t['Major'].str.replace('))', 'nnn', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace(')', '', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace('nnn', ')', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace('(', '', regex=False)
qualifs_t['Qualification'] = qualifs_t['Qualification'].str.strip()
qualifs_t['Major'] = qualifs_t['Major'].str.strip()
qualifs_t = qualifs_t.drop(columns=['column', 'qualif'])

# Export to a new file
qualifs_t.to_csv('cleaned_data\qualifications_table.csv')
qualifs_t.head()

,u_id,Qualification,Major
0,0,Bachelor of Science,Communications
1,1,Bachelor of Arts,Finance
2,2,Certificate,Educating/Teaching Adult
3,3,Bachelor,finance & Mgt
4,4,Bachelor,Economics


### Other Experience

`['Other Experience']` is another column that contains multiple pieces of data that could be extracted. This next section will show extracting number of years teaching experience and number of years professional experience.

#### Teaching Experience

These steps start by cleaning up the Other Experience column to make it easier to work with.

In [19]:
# Clean Other Experience - Teaching Experience
# Make all lower case
data['Other Experience'] = data['Other Experience'].str.lower()

# Remove double spaces
data['Other Experience'] = data['Other Experience'].str.replace('  ', ' ')

Next, the column is split up using common phrases among the data in the column that is either prefixed or suffixed with a number for the years. These split up columns are then stripped of special symbols and uses rules to identify those with a discovered number of years.

In [20]:
# Split column by common phrasing
teaching_experience = data['Other Experience'].str.split('years of high school|'
                                                         'years high school|'
                                                         'years school|'
                                                         'years of school|'
                                                         'years university|'
                                                         'years of university|'
                                                         'years teaching experience|'
                                                         'years of teaching experience|'
                                                         'years teaching|'
                                                         'years of teaching', expand=True)

# Strip out irrelevant symbols
teaching_experience[0] = teaching_experience[0].str.replace('+', '', regex=True)
teaching_experience[0] = teaching_experience[0].str.replace('\n', '', regex=True)
teaching_experience[0] = teaching_experience[0].str.strip()
teaching_experience[1] = teaching_experience[1].str.replace('\n', '', regex=True)
teaching_experience[1] = teaching_experience[1].str.replace(':', '')
teaching_experience[1] = teaching_experience[1].str.replace('&', '')
teaching_experience[1] = teaching_experience[1].str.strip()

# Extract numbers from columns
teaching_experience['n'] = teaching_experience[0].str.split(' ').str[-1]
teaching_experience['n2'] = teaching_experience[1].str.split(' |y').str[0]

# Combine isolated numerical values
teaching_experience['years'] = np.where(teaching_experience[0].str.isdigit(), teaching_experience[0],
                               np.where(teaching_experience['n'].str.isdigit(), teaching_experience['n'],
                               np.where(teaching_experience['n2'].str.isdigit(), teaching_experience['n2'],
                               None)))

# Convert column to numeric
teaching_experience['years'] = pd.to_numeric(teaching_experience['years'])

# Remove outliers
teaching_experience['years'] = np.where(teaching_experience['years'] > 40, None, teaching_experience['years'])

teaching_experience.head()

,0,1,2,n,n2,years
0,"10 years small business ownership, 10 years sm...",,None,8,,8.0
1,,17 years of professional/management ex...,None,,17,17.0
2,,15years of professional/management experience 3,None,,15,15.0
3,6 years professional experience 6,and active researcher,None,6,and,6.0
4,1 year professional experience 24,and active researcher. curriculum development ...,None,24,and,24.0


Now the column can be returned to the original dataset and examined for data quality.

In [21]:
# Add column to original dataset
data['years_teaching'] = teaching_experience['years']

# Check for nulls
missing = data['years_teaching'].isnull().sum()
print("\nNumber of missing values:", missing)


Number of missing values: 148


The column is still missing quite a few number of values. This could be for two reasons:
- The previous logic did not pick up the number of years
- The student had no previous experience teaching

To address the first problem, the `['Other Experience']` column is checked to see if it contains `'teach'`. If it is populated with the mean value of the years taught.
For the second problem, any staff without this keyword we will put the value as 0 and assume the student has no experience.

In [22]:
# Calculate the mean
mean = round(np.mean(pd.to_numeric(data['years_teaching'])))

# Correct the column
data['years_teaching'] = np.where(data['Other Experience'].str.contains('teach') &
                                  data['years_teaching'].isna(), mean,
                         np.where(data['years_teaching'].isna(), 0, data['years_teaching']))

# Ensure column is all the same data type
data['years_teaching'] = pd.to_numeric(data['years_teaching'])

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,Criteria,t_business,t_management,t_account,t_finance,t_market,Title_clean,HQL_clean,HQ_clean,years_teaching
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,Industry & Teaching experience,1,1,0,0,0,Business,Masters,Master of Arts,8.0
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,Certification,0,1,1,0,0,Business,Masters,Master of Science,17.0
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,Industry & Teaching experience,1,0,0,0,1,Business,Masters,Master of Business Administration,15.0
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,Researcher,0,1,0,0,0,Business,Doctorate,Ph.D,6.0
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,Researcher/Curriculum Development,1,0,0,0,0,Business,Doctorate,Ph.D,24.0


#### Professional Experience
A similar process can be taken for number of years professional experience, as shown below.

In [23]:
# Clean Other Experience - Professional Experience
# Split column by common phrasing
prof_experience = data['Other Experience'].str.split('years professional experience|'
                                                     'years of professional experience|'
                                                     'years professional|'
                                                     'years of professional|'
                                                     'years work|'
                                                     'years industry|'
                                                     'years of industry', expand=True)

# Strip out irrelevant symbols
prof_experience[0] = prof_experience[0].str.replace('+', '', regex=True)
prof_experience[0] = prof_experience[0].str.replace('\n', '', regex=True)
prof_experience[0] = prof_experience[0].str.strip()
prof_experience[1] = prof_experience[1].str.replace('\n', '', regex=True)
prof_experience[1] = prof_experience[1].str.replace(':', '')
prof_experience[1] = prof_experience[1].str.replace('&', '')
prof_experience[1] = prof_experience[1].str.strip()

# Extract numbers from columns
prof_experience['n'] = prof_experience[0].str.split(' ').str[-1]
prof_experience['n2'] = prof_experience[1].str.split(' |y').str[0]

# Combine isolated numerical values
prof_experience['years'] = np.where(prof_experience[0].str.isdigit(), prof_experience[0],
                               np.where(prof_experience['n'].str.isdigit(), prof_experience['n'],
                               np.where(prof_experience['n2'].str.isdigit(), prof_experience['n2'],
                               None)))

# Convert column to numeric
prof_experience['years'] = pd.to_numeric(prof_experience['years'])

# Remove outliers
prof_experience['years'] = np.where(prof_experience['years'] > 40, None, prof_experience['years'])

# Add column to original dataset
data['years_prof'] = prof_experience['years']

# Check for nulls
missing = data['years_prof'].isnull().sum()
print("\nNumber of missing values:", missing)

# Calculate the mean
mean = round(np.mean(pd.to_numeric(data['years_prof'])))

# Correct the column
data['years_prof'] = np.where(data['Other Experience'].str.contains('prof') &
                                  data['years_prof'].isna(), mean,
                         np.where(data['years_prof'].isna(), 0, data['years_prof']))

# Ensure column is all the same data type
data['years_prof'] = pd.to_numeric(data['years_prof'])

data.head()


Number of missing values: 159


,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,t_business,t_management,t_account,t_finance,t_market,Title_clean,HQL_clean,HQ_clean,years_teaching,years_prof
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,1,1,0,0,0,Business,Masters,Master of Arts,8.0,0
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,0,1,1,0,0,Business,Masters,Master of Science,17.0,17
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,1,0,0,0,1,Business,Masters,Master of Business Administration,15.0,15
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,0,1,0,0,0,Business,Doctorate,Ph.D,6.0,6
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,1,0,0,0,0,Business,Doctorate,Ph.D,24.0,10


#### Active Researcher
An extra piece of information is extracted to determine whether or not the student had experience as an active researcher using found key words in the `['Other Experience']` column.

In [24]:
# Add column for active researcher
data['active_researcher'] = np.where(data['Other Experience'].str.contains('active') &
                                     data['Other Experience'].str.contains('research'), 1, 0)

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,t_management,t_account,t_finance,t_market,Title_clean,HQL_clean,HQ_clean,years_teaching,years_prof,active_researcher
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,1,0,0,0,Business,Masters,Master of Arts,8.0,0,0
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,1,1,0,0,Business,Masters,Master of Science,17.0,17,0
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,0,0,0,1,Business,Masters,Master of Business Administration,15.0,15,0
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,1,0,0,0,Business,Doctorate,Ph.D,6.0,6,1
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,0,0,0,0,Business,Doctorate,Ph.D,24.0,10,1


## Converting Cleaned Columns to Machine Readable

The most popular machine learning package is sci-kit, which requires data to be fed into it in order to create models. Steps left to complete are:

- Normalise numeric columns
- Encode categorical columns

This section will go over an example final data set to be used for machine learning. Potentially relevant columns will be preprocessed, and any other columns are removed.

With the final table, uneeded columns are removed and missing values are examined once again.

In [25]:
# Remove remaining redundant columns
data = data.drop(columns=['ID', 'Name', 'Title', 'Join Date', 'LWD',
                          'Highest Qualification Level', 'Highest Qualification', 'HQ_clean',
                          'Major', 'University', 'All Qualifications', 'Courses Taught',
                          'MAJOR TEACHING FIELD', 'Other Experience', 'Criteria'])

# Count Missing
count_missing = data.isnull().sum()
total = len(data)

percent_missing = (count_missing/total) * 100
print(percent_missing)

u_id                 0.000000
Location             4.929577
Reports To           1.408451
t_business           0.000000
t_management         0.000000
t_account            0.000000
t_finance            0.000000
t_market             0.000000
Title_clean          0.000000
HQL_clean            0.000000
years_teaching       0.000000
years_prof           0.000000
active_researcher    0.000000
dtype: float64


Since the percentage missing of the remaining columns is low, we can drop the empty rows.

In [26]:
# Remove null rows
data = data.dropna()


### Normalise Numerical Columns
This is the first time this will use the `sklearn` package. Within it, the mix max scaling technique is used to normalise the two numerical columns, `['years_teaching']` and `['years_prof']`, in the dataset.

In [27]:
# Normalise Numerical Columns
# Years Teaching
years_teaching = data['years_teaching'].values
years_teaching = years_teaching.reshape(-1, 1)
years_teaching_n = preprocessing.MinMaxScaler().fit_transform(years_teaching)
data['years_teaching_n'] = pd.DataFrame(years_teaching_n)

# Years Professiona
years_prof = data['years_prof'].values
years_prof = years_prof.reshape(-1, 1)
years_prof_n = preprocessing.MinMaxScaler().fit_transform(years_prof)
data['years_prof_n'] = pd.DataFrame(years_prof_n)

# Remove no longer required columns
data = data.drop(columns=['years_teaching', 'years_prof'])

data.head()

,u_id,Location,Reports To,t_business,t_management,t_account,t_finance,t_market,Title_clean,HQL_clean,active_researcher,years_teaching_n,years_prof_n
0,0,London,Abbas Houda,1,1,0,0,0,Business,Masters,0,0.235294,0.000000
1,1,London,Abbas Houda,0,1,1,0,0,Business,Masters,0,0.500000,0.500000
2,2,London,Abbas Houda,1,0,0,0,1,Business,Masters,0,0.441176,0.441176
3,3,London,Abbas Houda,0,1,0,0,0,Business,Doctorate,1,0.176471,0.176471
4,4,London,Tony Emmanuel,1,0,0,0,0,Business,Doctorate,1,0.705882,0.294118


### Encoding String Columns

For each column, the values are first converted into indexes in order to keep the future column names clean. Then the dummy columns are generated and added to the original data set.

In [28]:
# Encoding String Columns
# Create dictionaries

# Location
location_list = list(set(data['Location'].dropna().to_list()))
location_dict = {k: v for v, k in enumerate(location_list)}
print(location_dict)
# Reports To
reports_list = list(set(data['Reports To'].dropna().to_list()))
reports_dict = {k: v for v, k in enumerate(reports_list)}
print(reports_dict)
# Title
title_list = list(set(data['Title_clean'].dropna().to_list()))
title_dict = {k: v for v, k in enumerate(title_list)}
print(title_dict)
# Highest Qualification Level
hql_list = list(set(data['HQL_clean'].dropna().to_list()))
hql_dict = {k: v for v, k in enumerate(hql_list)}
print(hql_dict)
# Replace Columns
data = data.replace({"Location": location_dict})
data = data.replace({"Reports To": reports_dict})
data = data.replace({"Title_clean": title_dict})
data = data.replace({"HQL_clean": hql_dict})

# Create dummy dataframes
loc_dummies = pd.get_dummies(data['Location'], prefix='Location')
rep_dummies = pd.get_dummies(data['Reports To'], prefix='Reports To')
tit_dummies = pd.get_dummies(data['Title_clean'], prefix='Title')
hql_dummies = pd.get_dummies(data['HQL_clean'], prefix='HQL')

loc_dummies.head()
rep_dummies.head()
tit_dummies.head()
hql_dummies.head()

# Join dummies back to the original dataset
data = pd.concat([data, loc_dummies, rep_dummies, tit_dummies, hql_dummies], axis=1)

# Remove no longer required columns
data = data.drop(columns=['Location', 'Reports To', 'Title_clean', 'HQL_clean'])

data.head()

{'Belfast': 0, 'Bolton': 1, 'Southampton': 2, 'Bristol': 3, 'Leeds': 4, 'Manchester': 5, 'Leicester': 6, 'London': 7, 'Birmingham': 8, 'Oxford': 9, 'Cambridge': 10, 'Liverpool': 11, 'Cardiff': 12, 'Glasgow': 13, 'Brighton': 14}
{'Kim Hughes': 0, 'Chris Peters': 1, 'Jayne Radford': 2, 'Sean Woods': 3, 'Ian Varley': 4, 'Orlamei Princess': 5, 'Tony Emmanuel': 6, 'Nick Omotayo': 7, 'Betty Adewuyi': 8, 'Harriet Mensah': 9, 'Rob Jenkins': 10, 'Abbas Houda': 11, 'Ian Westwood': 12, 'Daphne Keaton': 13, 'Olu Ebenezer': 14}
{'Business (HR & Admin)': 0, 'Business (Admin)': 1, 'Business': 2, 'Trainee': 3, 'Business & Comp Lit': 4, 'Business, Comp & Math': 5}
{'Doctorate': 0, 'Bachelors': 1, 'Masters': 2}


,u_id,t_business,t_management,t_account,t_finance,t_market,active_researcher,years_teaching_n,years_prof_n,Location_0,...,Reports To_14,Title_0,Title_1,Title_2,Title_3,Title_4,Title_5,HQL_0,HQL_1,HQL_2
0,0,1,1,0,0,0,0,0.235294,0.000000,0,...,0,0,0,1,0,0,0,0,0,1
1,1,0,1,1,0,0,0,0.500000,0.500000,0,...,0,0,0,1,0,0,0,0,0,1
2,2,1,0,0,0,1,0,0.441176,0.441176,0,...,0,0,0,1,0,0,0,0,0,1
3,3,0,1,0,0,0,1,0.176471,0.176471,0,...,0,0,0,1,0,0,0,1,0,0
4,4,1,0,0,0,0,1,0.705882,0.294118,0,...,0,0,0,1,0,0,0,1,0,0


The final table gives a fully processed dataset of potential features for each student.

The new clean data is now saved to file and used for a Machine Learning model.

In [29]:
# Save to file
data.to_csv('cleaned_data\student_table.csv')